# Part 9: Vitis Accel

In the previous sections we've seen how to deploy a model on a [pynq-z2 board](http://www.pynq.io/) with the `VivadoAccelerator` backend. In this section, we introduce the `VitisAccelerator` backend of `hls4ml`, an alternative workflow for deploying models that uses AMD's Vitis Accel workflow. This time, we will deploy the model on an [Alveo U55C Accelerator Card](https://www.xilinx.com/products/boards-and-kits/alveo/u55c.html).

In [ ]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
import os

# os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

## Load model
Load the model from `part4: quantization` (note you need to have trained the model in part 4 first)

In [ ]:
model = load_model('model_3/KERAS_check_best_model.h5', custom_objects=co)

## Convert to hls4ml
Similar to part 7, We'll convert our model into `hls4ml`, targeting the `backend='VitisAccelerator'` backend this time around. This backend wraps the HLS model and adds a CPU/MCU program that drives the FPGA. We also specify `board='alveo-u55c'`.

In [ ]:
import sys
import os

local_module_dir = os.path.join("/home/jovyan/work/hls4ml") # This should be changed the path of the local modified version of HLS4ML containing Vitis Accel backend
sys.path.insert(0, local_module_dir)

import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'

print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config,
    output_dir='model_3/hls4ml_prj_vitis_accel',
    backend='VitisAccelerator',
    board='alveo-u55c',
    num_kernel=4,
    num_thread=8,
    batchsize=8192
)
hls_model.compile()

We can see which baords are supported in the [documentation](https://fastmachinelearning.org/hls4ml/advanced/accelerator.html). The `VitisAccelerator` backend has a different `AcceleratorConfig` section of the configuration. Here we can change some details of the Vitis Accel project.

In this example, we set `'num_kernel=4'`, directing Vitis Accel to create 4 copies of the kernel on the FPGA. We also set `'num_thread=8'`, meaning that the host code will create 8 threads to drive the FPGA. For a multi-core CPU, this improves throughput by increasing the speed at which data is written to the FPGA memory.

The `create_initial_config` method (of any backend) can be used to create a template dictionary with the default parameters that you can use as a starting point.

In [ ]:
plotting.print_dict(hls4ml.backends.get_backend('VitisAccelerator').create_initial_config())

## Predict
Run the CPU emulation of the hls4ml NN and save the file to compare against the hardware result later.

In [ ]:
import numpy as np

X_test = np.load('X_test.npy')
y_hls = hls_model.predict(np.ascontiguousarray(X_test))
np.save('model_3/y_hls.npy', y_hls)